## Day 34 Lecture 1 Assignment

In this assignment, we will learn about SVR. We will use the absenteeism at work dataset loaded below and analyze the model generated for this dataset.

In [0]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore')

In [0]:
absent = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Absenteeism_at_work.csv', sep=';')

In [0]:
absent.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,97,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,97,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,97,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,2


In [0]:
absent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 21 columns):
ID                                 740 non-null int64
Reason for absence                 740 non-null int64
Month of absence                   740 non-null int64
Day of the week                    740 non-null int64
Seasons                            740 non-null int64
Transportation expense             740 non-null int64
Distance from Residence to Work    740 non-null int64
Service time                       740 non-null int64
Age                                740 non-null int64
Work load Average/day              740 non-null float64
Hit target                         740 non-null int64
Disciplinary failure               740 non-null int64
Education                          740 non-null int64
Son                                740 non-null int64
Social drinker                     740 non-null int64
Social smoker                      740 non-null int64
Pet                          

Recall that we removed all correlated columns as well as columns that are a function of two or more other columns (like BMI and weight and height) as well as columns that are not usefu to us (like ID).
We also created dummy variables for the categorical columns in our dataset.
Finally, we split the data into test and train (with test being 20% of the data).

Perform these actions below.

In [0]:
#answer below:
drops = ['Disciplinary failure', 'Body mass index', 'Hit target', 
         'Work load Average/day ', 'Month of absence', 'Service time', 
         'ID', 'Weight', 'Social smoker', 'Seasons', 'Reason for absence',
         'Day of the week']

df = absent.drop(columns=drops, axis=1)

df = pd.concat([df, pd.get_dummies(absent['Day of the week'], prefix='weekday', 
                                   drop_first=True),
                pd.get_dummies(absent['Reason for absence'], prefix='reason_code', 
                               drop_first=True)], 
               axis=1)

df.columns = [x.lower().replace(' ', '_') for x in df.columns]

X = df.drop(columns='absenteeism_time_in_hours', axis=1)
y = df['absenteeism_time_in_hours']

Scale the indepdendent variables using the standard scaler

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

cols = X.columns
scaler = StandardScaler()

X = scaler.fit_transform(X)
X = pd.DataFrame(X, columns=cols)

Split the data into train and test (test proportion=0.2).

In [0]:
# answer below:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,
                                                    random_state=42)


Generate an SVR model to predict the number of hours of absenteeism. Compare the train and test scores.

In [0]:
# answer below:
model = SVR()

model.fit(X_train, y_train)

y_predict = model.predict(X_test)

print(f'Training score: {model.score(X_train, y_train)}')
print(f'Test score: {model.score(X_test, y_test)}')



Training score: 0.08864735667530665
Test score: 0.10421615095922698


If you are unhappy with your model's performance, perform a grid search to optimize C.

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 40 columns):
transportation_expense             740 non-null int64
distance_from_residence_to_work    740 non-null int64
age                                740 non-null int64
education                          740 non-null int64
son                                740 non-null int64
social_drinker                     740 non-null int64
pet                                740 non-null int64
height                             740 non-null int64
absenteeism_time_in_hours          740 non-null int64
weekday_3                          740 non-null uint8
weekday_4                          740 non-null uint8
weekday_5                          740 non-null uint8
weekday_6                          740 non-null uint8
reason_code_1                      740 non-null uint8
reason_code_2                      740 non-null uint8
reason_code_3                      740 non-null uint8
reason_code_4                  

In [0]:
# answer below:
from sklearn.model_selection import GridSearchCV

params = {'kernel':['rbf', 'poly', 'linear'],
          'C':[.1, .25, .5, .75, 1],
          'gamma':['auto', 'scale', .0025, .025],
          'epsilon':[.005, .0075, .01, .0125, .015]}

model = GridSearchCV(SVR(), param_grid=params, cv=5) 

In [0]:
#from sklearn.model_selection import GridSearchCV 
#from sklearn.svm import SVR 

#gsc = GridSearchCV( estimator=SVR(kernel='rbf'), 
#                  param_grid={ 'C': [0.1, 1, 100, 1000], 
#                              'epsilon': [0.0001, 0.0005, 0.001, 0.005, 0.01,
                #                            0.05, 0.1, 0.5, 1, 5, 10], 
                #                'gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5] }, 
                #    cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

# grid_result = gsc.fit(X, y) 
# best_params = grid_result.best_params_ 
# best_svr = SVR(kernel='rbf', C=best_params["C"], epsilon=best_params["epsilon"], 
#                gamma=best_params["gamma"], coef0=0.1, shrinking=True, tol=0.001, 
#                cache_size=200, verbose=False, max_iter=-1)

In [0]:
model_result = model.fit(X_train, y_train)
best_params = model_result.best_params_
best_svr = SVR(kernel=best_params['kernel'], C=best_params['C'], 
               epsilon=best_params['epsilon'], gamma=best_params['gamma'])


In [0]:
best_svr.fit(X_train, y_train)

print(f'Training score: {best_svr.score(X_train, y_train)}')
print(f'Test score: {best_svr.score(X_test, y_test)}')

Training score: 0.11055345079152812
Test score: 0.07457257711152776


In [0]:
best_params

{'C': 0.75, 'epsilon': 0.015, 'gamma': 'auto', 'kernel': 'linear'}

In [0]:
params = {'C':[.75, .775, 1, 1.25, 1.5],
          'epsilon':[.015, .0175, .1, .125, .15]}

model = GridSearchCV(SVR(kernel='linear', gamma='auto'), param_grid=params, cv=5) 
model_result = model.fit(X_train, y_train)
best_params = model_result.best_params_
best_svr = SVR(kernel='linear', C=best_params['C'], 
               epsilon=best_params['epsilon'], gamma='auto')

In [0]:
best_svr.fit(X_train, y_train)

print(f'Training score: {best_svr.score(X_train, y_train)}')
print(f'Test score: {best_svr.score(X_test, y_test)}')

Training score: 0.1117138302003452
Test score: 0.07659422290087259


In [0]:
best_params

{'C': 1, 'epsilon': 0.1}